In [1]:
import requests
import csv
import re
from bs4 import BeautifulSoup
import pandas as pd
import json
from selenium import webdriver
from gspread_dataframe import set_with_dataframe

In [2]:
url = 'https://www.tradeindia.com/Seller/'
categories=pd.DataFrame()
x=0

res = requests.get(url)
soup = BeautifulSoup(res.text,"lxml")
category_html = soup.find_all("div",class_="bx--col-sm-4 bx--col-md-4 bx--col-lg-4 catBoxMain")
cat_html = soup.find_all('a', class_="cat-name d-block pt-10")

top = len(soup.find_all('a', class_="cat-name d-block pt-10"))

In [3]:
%%time
## take all categories availables
df0 = pd.DataFrame()

for n in range(0,top):
    
    df0.loc[x,'category'] = getattr(cat_html[n],'text').replace('\n','')
    
    ### clean category name
    df0.category = df0.category.str.replace(' & ','-')
    df0.category = df0.category.str.replace(',','-')
    df0.category = df0.category.str.replace(' ','-')
    df0.category = df0.category.str.replace('\n','')
    
    x+=1
    

Wall time: 72.4 ms


In [4]:
df0['url'] = 'https://www.tradeindia.com/Seller/'+df0.category+'/'

In [9]:
%%time

df = pd.DataFrame()
x = 0 
for n in range(0,len(df0)):

    res2 = requests.get(df0.url[n])

    soup2 = BeautifulSoup(res2.text,"lxml")

    subcat2 = soup2.find_all('a', class_="cat-name d-block pt-10")

    for i in range(0,len(subcat2)):
        
        df.loc[x,'url'] = df0.url[n]
        
        df.loc[x,'category'] = df0.category[n]
        
        df.loc[x,'subcat'] = getattr(subcat2[i],'text')
        
        df.subcat = df.subcat.str.replace(' & ','-')
        df.subcat = df.subcat.str.replace(' ','-')
        
        
        x+=1

Wall time: 1min 39s


In [12]:
df['url_final'] = df.url +df.subcat+'/'

In [18]:
df.url_final.head(1).to_list()
### in this example i only take the first final url (rice)

['https://www.tradeindia.com/Seller/Agriculture/Rice/']

In [24]:
%%time

## take all products in this subcategory
product = pd.DataFrame()
row = 0

for n in df.url_final.head(1).to_list():
    
    res_sub = requests.get(n)
    
    soup_sub = BeautifulSoup(res_sub.text,"lxml")
    
    base = soup_sub.find_all("a",class_="dark-link")

##----
    for i in range(0,len(base)):
        
        product.loc[row,'url'] = n
        
        #print(n)
        
        product.loc[row,'name'] = getattr(base[i],'text')
        
        product.name = product.name.str.replace(' & ','-')
        product.name = product.name.str.replace(' ','-')
        
        product.name = product.name.str.lower()
        
        product['final_url'] = 'https://www.tradeindia.com/manufacturers/'+product.name+'.html'
        
        product = product.reset_index(drop=True)
        
        #print(getattr(base[i],'text'))
        
        row+=1

Wall time: 2.67 s


In [99]:
pagination = pd.DataFrame()
row = 0

for n in range(0,len(product)):
    res_pag = requests.get(product.final_url[1],allow_redirects=False)
    soup_pag = BeautifulSoup(res_pag.text,"lxml")
    base2_pag = soup_pag.find_all("li",class_="a-normal")
    try:
        pagination.loc[row,'url'] = (product.final_url[n])
        #print(product.final_url[n])
        pagination.loc[row,'pag'] = getattr(base2_pag[1],'text')
        #print(getattr(base2_pag[1],'text'))
    except:
        pagination.loc[row,'url'] = (product.final_url[n])
        #print(product.final_url[n])
        pagination.loc[row,'pag'] = 1        
    
    print(n)
    row+=1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82


2

In [98]:
getattr(base2_pag[1],'text')

'3'

In [ ]:
#product2.url[429]
product2 = total_pages
top_total = len(total_pages)
total_pages.pag = total_pages.pag.fillna(1)
total_pages

In [ ]:
 for x in range(0,top_pages):
        top_pagination = product2.pag[x]
        for i in range(1,top_pagination):
            res_final = requests.get(total_pages.url[x]+'?page='+[i],allow_redirects=False)
            print(total_pages.url[x]+'?page='+[i])
            soup_final = BeautifulSoup(res_final.text,"lxml")
            base_final = soup_final.find_all("div", class_="product-specification")  
        
        
        